This script is designed to schedule a time-series microcosm experiment in which all microcosms are destructively sampled at the same time, rather than sequentially from t=0. 

REQUIRED ARGUMENTS

    -s     a start date and time [format: ]

    -c     the e-mail address associated with the google calendar
                    
                
REQUIREMENTS
    gcalcli     https://github.com/insanum/gcalcli
    

In [ ]:
import os, re, sys

In [ ]:
START = ''
EMAIL = ''

# Read command line args
myopts, args = getopt.getopt(sys.argv[1:],"s:c:")

for o, a in myopts:
    if o == '-s':
        START= a
    if o == '-c':
        EMAIL= a

